In [1]:
import requests, math, re, sys, time, json
from bs4 import  BeautifulSoup as bs

In [3]:
rs = requests.session()
res = rs.get('http://vacation.eztravel.com.tw/pkgfrn/results/TPE/93?fullStatus=NONE')
soup = bs(res.text, "lxml")

In [6]:
count  = soup.select('.v-line')[0].text.split(' ')
print count[1]
page = int(math.ceil(float(count[1]) / 18))
print page

186
11


In [7]:
urlFormat = 'http://vacation.eztravel.com.tw/pkgfrn/results/TPE/93/{}?&fullStatus=NONE'
with open('E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Kanto.txt','w') as f:
    #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Kanto.txt
    #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Link\\bid_list_for_EZtravel_in_Kanto.txt
    for pageNum in range(1, page + 1):
        url = urlFormat.format(pageNum)
        res2 = rs.get(url)
        soup1 = bs(res2.text, "lxml")
        list_table = soup1.select('div#listTable')[0]
        for items in list_table.select('div.list-box.mainLinkBox'):
            f.write('http://vacation.eztravel.com.tw/pkgfrn/' + items.select('a')[0]['href'].split('/pkgfrn/')[1] + '\n')

In [8]:
dicCheck = {} #宣告一個dic，稍後會將檔名(Key)跟網址(Value)一一放入
num = 0
for bid_url in open('E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Kanto.txt','r'): #讀連結檔
    #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Kanto.txt
    #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Link\\bid_list_for_EZtravel_in_Kanto.txt
    num += 1
    splitStr = bid_url.split('/') #將網址切割，並且丟入splitStr這個List裡
    linkName = splitStr[5].strip() + "_" + splitStr[6].strip() #將List[index]對應到的元素，以字串型式組合成dataNmae
    #用商品編號當作檔名
    if linkName not in dicCheck:
        print 'No.' + str(num) + ', ' + linkName +  ' is new, and added in dicCheck.'
        dicCheck[linkName] = bid_url
    else:
        print 'No.' + str(num) + ', ' + linkName + ' is existed, and not added in dicCheck.'
        print linkName, dicCheck[linkName] #若有重複，則print出重複的Key跟Value

No.1, VDR0000001943_NRT05IT6612C is new, and added in dicCheck.
No.2, VDR0000007986_TYO05IT60216A_6699 is new, and added in dicCheck.
No.3, VDR0000007986_IBR05ZV60516A_6415 is new, and added in dicCheck.
No.4, VDR0000001838_NTE061005IT6 is new, and added in dicCheck.
No.5, FRN0000013840_20160602 is new, and added in dicCheck.
No.6, VDR0000007614_JTP100160603IT is new, and added in dicCheck.
No.7, VDR0000001838_NTX061405BR6F is new, and added in dicCheck.
No.8, VDR0000001838_NTE061805BR6 is new, and added in dicCheck.
No.9, VDR0000001943_NRT05IT6606A is new, and added in dicCheck.
No.10, VDR0000007986_TYO05GE61116E_5520 is new, and added in dicCheck.
No.11, VDR0000001838_NTX061405BR6J is new, and added in dicCheck.
No.12, FRN0000010576_20160602 is new, and added in dicCheck.
No.13, VDR0000001943_NRT05CX6604C is new, and added in dicCheck.
No.14, VDR0000007986_TYO05GE60716G_5520 is new, and added in dicCheck.
No.15, FRN0000010667_20160602 is new, and added in dicCheck.
No.16, FRN00000132

In [9]:
### 印出字典的key與value########
def PrintKeyValue(dic_in):
    for key, value in dic_in.iteritems():
        print key, ':', value 

In [10]:
itemDetail = {} #放置所有細項資料集合
eachTravel = [] #放置所有細項資料於旅行方案集合
totalTravel = {} #放置所有的旅行方案於總集合
totalCount = 0
errorCount = 0
with open('E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Data\\Error\\error_list_for_EZtravel_in_Kanto.txt', 'w') as errorFile:
    #寫error檔
    #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Data\\Error\\error_list_for_EZtravel_in_Kanto.txt
    #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Data\\Error\\error_list_for_EZtravel_in_Kanto.txt
    for bid_url in open('E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Kanto.txt', 'r'):
        #讀連結檔
        #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Kanto.txt
        #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Link\\bid_list_for_EZtravel_in_Kanto.txt
        try: #使用try-except
            daysHotFormat = 'day{}_Hotel' #建立字串格式
            daysTourFormat = 'day{}_Tour' #同上
            bid_detail = requests.get(bid_url.strip()) #讀取檔案中的連結
            soup2 = bs(bid_detail.text, "lxml") #萃取資料
            row = soup2.select('div#pkgfrnVisitHistory')[0]
            price = re.search('\d+', row.select('.price')[0].text) #利用正規表示法抓取價格
            plane = re.search('.\W{2,4}', row.select('div#flight-block1 span')[3].text) #利用正規表示法抓取航空公司名稱
            daysTransfer = (row.select('div.product-info.css-td')[0].text.strip().split(u'天數')) #抓每筆旅行方案的旅行天數
            days = (re.search('\d{1}', daysTransfer[1])) #利用正規表示法抓出旅行天數
            itemDetail['Title'] = row.select('div.pro-title.css-td h1')[0].text #將個別資料丟入itemDetail字典中，以下相同
            itemDetail['Price'] = price.group(0)
            itemDetail['City'] = row.select('.tag')[0].text.strip()
            itemDetail['Time'] = row.select('div#flight-block1 span')[2].text.strip()
            itemDetail['PlaneCom'] = plane.group(0)

            if len(row.select('div.pro-id-right.css-td span')[0].text.strip()) > 0: #判斷若PID_1長度大於零，則將資料丟入
                itemDetail['PID_1'] = row.select('div.pro-id-right.css-td span')[0].text.strip()
            else: #判斷若PID_1長度小於零，則丟入Null
                itemDetail['PID_1'] = 'Null'
            if len(row.select('div.pro-id-right.css-td span')[1].text.strip()) > 0: #同上
                itemDetail['PID_2'] = row.select('div.pro-id-right.css-td span')[1].text.strip()
            else: #同上
                itemDetail['PID_2'] = 'Null'
            totalCount += 1 #計算有多少筆資料丟入JSON檔中

            print '----------------------------這是分隔線----------------------------'
            print 'days: ' + days.group(0) #印出當前旅行方案的旅行天數
            print 'Url: ' + bid_url.strip() #印出當前旅行方案的網址

            #因為EZtravel的網站資料較鬆散，且只有旅館跟行程有規律性，故特別獨立出來寫加入的方法
            num = 1 #因住宿飯店的索引值是[1]、[3]、[5]……的奇數，所以給起始值，每執行一次會增加2
            for times in range(0,int(days.group(0))):
                daysHotInput = daysHotFormat.format(times + 1) #這邊是天數，day{1}_Hotel、day{2}_Hotel……
                itemDetail[daysHotInput] = row.select('div.meals')[num].text.strip()
                #這邊是索引值(奇數)，row.select('div.meals')[1].text.strip()、row.select('div.meals')[3].text.strip()……
                num += 2
            for indexTour in range(0,int(days.group(0))):
                daysTourInput = daysTourFormat.format(indexTour + 1) #這邊是天數，day{1}_Tour、day{2}_Tour……
                itemDetail[daysTourInput] = row.select('div.day-box h4')[indexTour].text.strip()
                #這邊是索引值，row.select('div.day-box h4')[1].text.strip()、row.select('div.day-box h4')[2].text.strip()……
            PrintKeyValue(itemDetail)
            eachTravel.append(itemDetail)
            totalTravel['eachTravel'] = eachTravel

        except:
            errorCount += 1
            splitPID = bid_url.strip().split('/')
            strPID = splitPID[5].strip() + "_" + splitPID[6].strip()
            #當錯誤訊息出現時，印出是哪一筆資料出現錯誤
            print '----------------' + strPID + ' is error----------------'
                #將出現錯誤的資料網址，寫入txt檔中
            errorFile.write(bid_url)

with open('E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Data\\totalTravel_for_EZtravel_in_Kanto.json', 'w') as dataFile:
    #寫最後全部資料檔
    #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Data\\totalTravel_for_EZtravel_in_Kanto.json
    #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Data\\totalTravel_for_EZtravel_in_Kanto.json
    json.dump(totalTravel, dataFile)

print '----------------------------All datas are done!!!----------------------------'
print 'Data Quantities: ' + str(totalCount)
print 'Error Quantities: ' + str(errorCount)

----------------------------這是分隔線----------------------------
days: 5
Url: http://vacation.eztravel.com.tw/pkgfrn/introduction/VDR0000001943/NRT05IT6612C
City : 福江
PID_1 : VDR0000001943
day4_Hotel : 愛宕山東急REI或同級
day5_Tour : 東京－成田機場－桃園機場
day4_Tour : 飯店－迪士尼
PlaneCom : 台灣虎航 
PID_2 : NRT05IT6612C
Price : 19900
Title : 前衛東京~明治神宮.遠眺晴空塔.時尚台場購物商場.半自由行五日
day1_Hotel : 上野CANDEO或同級
day3_Hotel : 千葉CANDEO、幕張海濱APA、幕張ＦＲＡＮＣＥ、東急REI愛宕山或同級
day2_Tour : 秋葉原－原宿（明治神宮－竹下通－原宿最新商城ALTA）－東京車站-飯店
day2_Hotel : 千葉CANDEO、幕張海濱APA、幕張ＦＲＡＮＣＥ、東急REI愛宕山或同級
Time : 14:50 - 19:00
day3_Tour : 飯店－築地市場－東京鐵塔One Piece－代官山(下午茶),自由之丘,渋谷－六本木森大樓夜景
day5_Hotel : 甜蜜的家或同級
day1_Tour : 桃園國際機場－東京成田國際空港－淺草－晴空塔－飯店
----------------------------這是分隔線----------------------------
days: 5
Url: http://vacation.eztravel.com.tw/pkgfrn/introduction/VDR0000007986/TYO05IT60216A_6699
City : 東京
PID_1 : VDR0000007986
day4_Hotel : 東京 或 千葉 或 橫濱：Peral系列飯店 或 Dormy Inn東京八丁堀溫泉飯店(S-DBL一中床+拉麵消夜餐) 或 Candeo系列飯店  或 東橫inn系列飯店 或 太陽道系列飯店 或同級
day5_Tour : 東京飯店→東京成田機場／台北
day4_T